# Assignment #2: Language models
Author: Pierre Nugues

## Objectives

The objectives of this assignment are to:
* Write a program to find n-gram statistics
* Compute the probability of a sentence
* Know what a language model is
* Write a short report of 1 to 2 pages on the assignment

## Submission

Once you have written all the missing code and run all the cells, you will submit your notebook to an automatic marking system. Do not erase the content of the cells as we will possibly check your programs manually.
The submission instructions are at the bottom of the notebook.

## Organization

* Each group will have to write Python programs to count unigrams, bigrams, and trigrams in a corpus of approximately one million words and to determine the probability of a sentence.
* You can test you regular expression using the regex101.com site
* Each student will have to write a short report of one to two pages and comment briefly the results. In your report, you must produce the tabulated results of your analysis as described below.

## Programming

### Imports

Some imports you may need. Add others as needed.

In [532]:
import bz2
import math
import os
import regex as re
import requests
import sys
from zipfile import ZipFile

### Collecting and analyzing a corpus

Retrieve a corpus of novels by Selma Lagerl&ouml;f from this URL:
<a href="https://github.com/pnugues/ilppp/blob/master/programs/corpus/Selma.txt">
    <tt>https://github.com/pnugues/ilppp/blob/master/programs/corpus/Selma.txt</tt>
</a>. The text of these novels was extracted
from <a href="https://litteraturbanken.se/forfattare/LagerlofS/titlar">Lagerlöf arkivet</a> at
<a href="https://litteraturbanken.se/">Litteraturbanken</a>.

In [533]:
# You may have to adjust the path
corpus = open('./lab2/Selma.txt', encoding='utf8').read()

Run the <a href="https://github.com/pnugues/ilppp/tree/master/programs/ch02/python">concordance
program </a> to print the lines containing a specific word, for instance <i>Nils</i>.

In [534]:
pattern = 'Nils Holgersson'
width = 25

In [535]:
# spaces match tabs and newlines
pattern = re.sub(' ', r'\\s+', pattern)
# Replaces newlines with spaces in the text
clean_corpus = re.sub('\s+', ' ', corpus)
concordance = ('(.{{0,{width}}}{pattern}.{{0,{width}}})'
               .format(pattern=pattern, width=width))
for match in re.finditer(concordance, clean_corpus):
    print(match.group(1))
# print the string with 0..width characters on either side

Selma Lagerlöf Nils Holgerssons underbara resa genom Sv
! Se på Tummetott! Se på Nils Holgersson Tummetott!» Genast vände
r,» sade han. »Jag heter Nils Holgersson och är son till en husma
lden. »Inte är det värt, Nils Holgersson, att du är ängslig eller
 i dem. På den tiden, då Nils Holgersson drog omkring med vildgäs
ulle allt visa honom vad Nils Holgersson från Västra Vemmenhög va
om ägde rum det året, då Nils Holgersson for omkring med vildgäss
m vad det kan kosta dem. Nils Holgersson hade inte haft förstånd 
de det inte mer sägas om Nils Holgersson, att han inte tyckte om 
 Rosenbom?» För där stod Nils Holgersson mitt uppe på Rosenboms n
 Med ens fingo de syn på Nils Holgersson, och då sköt den store v
vila. När vildgässen och Nils Holgersson äntligen hade letat sig 
 slags arbetare. Men vad Nils Holgersson inte såg, det var, att s
nde han fråga, och om då Nils Holgersson sade nej, började han ge
de lille Mats, och om nu Nils Holgersson också hade tegat, så had
åg så försmädlig ut,

Run a simple <a href="https://github.com/pnugues/ilppp/tree/master/programs/ch05/python">tokenization
program</a> on your corpus.

In [536]:
def tokenize(text):
    words = re.findall('\p{L}+', text)
    return words

In [537]:
words = tokenize(corpus)
words[:10]

['Selma',
 'Lagerlöf',
 'Nils',
 'Holgerssons',
 'underbara',
 'resa',
 'genom',
 'Sverige',
 'Första',
 'bandet']

Count the number of unique words in the original corpus and when setting all the words in lowercase

Original text

In [538]:
# Write your code here
len(set(words))

44256

Lowercased text

In [539]:
# Write your code here
len(set([x.lower() for x in words]))

41032

### Segmenting a corpus

You will write a program to tokenize your text, insert `<s>` and `</s>` tags to delimit sentences, and set all the words in lowercase letters. In the end, you will only keep the words.

#### Normalizing 

Write a regular expression that matches all the characters that are neither a letter nor a punctuation sign. The punctuations signs will be the followings: `.;:?!`. In your regex, use the same order. For the definition of a letter, use a Unicode regex. You will call the regex string `nonletter`

In [540]:
# Write your code
#matches all chars that are not a letter or punctuation sign or spaces
nonletter = "[^\p{L};:?!.\s]" 

Write a `clean()` function that replaces all the characters that are neither a letter nor a punctuation sign with a space. The punctuations signs will be the followings: `.;:?!`.   For the sentence:

_En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa._

the result will be:

`En gång hade de på Mårbacka en barnpiga som hette Back Kajsa.`

In [541]:
# Write your code here
def clean(s):
    return re.sub(nonletter, "", s)


In [542]:
test_para = 'En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa. \
Hon var nog sina tre alnar lång, hon hade ett stort, grovt ansikte med stränga, mörka drag, \
hennes händer voro hårda och fulla av sprickor, som barnens hår fastnade i, \
när hon kammade dem, och till humöret var hon dyster och sorgbunden.'

In [543]:
test_para = clean(test_para)
test_para

'En gång hade de på Mårbacka en barnpiga som hette BackKajsa. Hon var nog sina tre alnar lång hon hade ett stort grovt ansikte med stränga mörka drag hennes händer voro hårda och fulla av sprickor som barnens hår fastnade i när hon kammade dem och till humöret var hon dyster och sorgbunden.'

#### Segmenter

In this section, you will write a sentence segmenter that will delimit each sentence with `</s>` and `<s>` symbols. For example the sentence:

_En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa._

will be bracketed as:

`<s> En gång hade de på Mårbacka en barnpiga som hette Back-Kajsa </s>`

As algorithm, you will use a simple heuristics to detect the sentence boundaries: A sentence starts with a capital letter and ends with a period-equivalent punctuation sign. You will write a regex to match these boundaries with a regular expression and you will insert `</s>\n<s>` symbols with a substitution function.

##### Detecting sentence boundaries

Write a regular expression that matches a punctuation, a sequence of spaces, and an uppercase letter. Call this regex string `sentence_boundaries`. In the regex, you will remember the value of the uppercase letter using a backreference. Use the Unicode regexes for the letters and the spaces.

In [544]:
# Write your code here
# matches the seq: uppercase letter, letters or space +, punctuation sign
sentence_boundaries = r"[.:;?!][\p{Z}\n]+([\p{Lu}])" 

##### Replacement markup

Write a string to replace the matched boundaries with the sentence boundary markup. Remember that a sentence ends with `</s>` and starts with `<s>` and that there is one sentence per line. Hint: The markup is `</s>\n<s>`. Remember also that the first letter of your sentence is in a regex backreference. Call the regex string `sentence_markup`.

In [545]:
# Write your code here
sentence_markup =  r" </s>\n<s> \1"

##### Applying the substitution

Use your regexes to segment your text. Use the string `sentence_boundaries`, `sentence_markup`, and `test_para` as input and `text` as output.

In [546]:
# Write your code here
def segmenter(text):
    p = re.compile(sentence_boundaries)
    text = p.sub(sentence_markup, text)
    text = "<s> "+text+"</s>"
    text = re.sub(r"[\n]+", " ", text)
    text = re.sub(r"[\.]", " ", text)
    text = re.sub(r"[\p{Z}]+", " ", text)
    return text.lower()

In [547]:
text = segmenter(test_para)
print(text)

<s> en gång hade de på mårbacka en barnpiga som hette backkajsa </s> <s> hon var nog sina tre alnar lång hon hade ett stort grovt ansikte med stränga mörka drag hennes händer voro hårda och fulla av sprickor som barnens hår fastnade i när hon kammade dem och till humöret var hon dyster och sorgbunden </s>


Insert markup codes in the beginning and end of the text

In [548]:
# Write your code here
def add_segment(text):
    return "<s> "+text+"</s>"

In [549]:
print(text)

<s> en gång hade de på mårbacka en barnpiga som hette backkajsa </s> <s> hon var nog sina tre alnar lång hon hade ett stort grovt ansikte med stränga mörka drag hennes händer voro hårda och fulla av sprickor som barnens hår fastnade i när hon kammade dem och till humöret var hon dyster och sorgbunden </s>


Replace the space duplicates with one space and remove the punctuation signs. For the spaces, use the Unicode regex.

In [550]:
# Write your code here
# removes duplicates spaces and also dots 
def space_dots_remover(text):
    text = re.sub(r"[\n]+", " ", text)
    text = re.sub(r"[\.]", " ", text)
    text = re.sub(r"[\p{Z}]+", " ", text)
    return text.lower()

In [551]:
text = space_dots_remover("<s> Hej    . . .  \n </s>")
print(text)

<s> hej </s>


Write a `segment_sentences(text)` function to gather the code in the Segmenter section and set the text in lowercase

In [552]:
# Write your code here
def segment_sentences(text):
    text = segmenter(text)
    return text


In [553]:
print(segment_sentences(test_para))

<s> en gång hade de på mårbacka en barnpiga som hette backkajsa </s> <s> hon var nog sina tre alnar lång hon hade ett stort grovt ansikte med stränga mörka drag hennes händer voro hårda och fulla av sprickor som barnens hår fastnade i när hon kammade dem och till humöret var hon dyster och sorgbunden </s>


Estimate roughly the accuracy of your program.

#### Tokenizing the corpus

Clean and segment the corpus

In [554]:
# Write your code here
# print(corpus[-558:])
corpus = clean(corpus)
corpus = segment_sentences(corpus)
print(corpus[-558:])

 <s> hon hade fått större kärlek av sina föräldrar än någon annan han visste och sådan kärlek måste vändas i välsignelse </s> <s> då prästen sade detta kom alla människor att se bort mot klara gulla och de förundrade sig över vad de såg </s> <s> prästens ord tycktes redan ha gått i uppfyllelse </s> <s> där stod klara fina gulleborg ifrån skrolycka hon som var uppkallad efter själva solen vid sina föräldrars grav och lyste som en förklarad </s> <s> hon var likaså vacker som den söndagen då hon gick till kyrkan i den röda klänningen om inte vackrare </s>


The result should be a normalized text without punctuation signs where all the sentences are delimited with `<s>` and `</s>` tags. The five last lines of the text should look like this.

In [555]:
print(corpus[-5000:])

d först karlarna och sedan kvinnfolken för att följa de två gamla ut till kyrkogården </s> <s> de som ställde upp sig hade en min på sig liksom om de förvånades över att de var med här </s> <s> för inte kände de någon sorg precis och inte heller ville de särskilt hedra någon av de döda </s> <s> det var bara så att när den nyheten hade spritt sig genom socknen att jan i skrolycka hade kommit till rätta just i passande tid för att kunna läggas i samma grav som kattrinna så hade alla tyckt att det var något vackert och märkvärdigt i detta och man hade velat vara med och se hur de gamla makarna förenades i döden </s> <s> man hade ju inte kunnat tro att så många andra skulle få samma tanke </s> <s> nu blev det nästan för stor uppståndelse för ett par så fattiga och ringa människors skull </s> <s> folk såg på varandra och skämdes smått men när de nu var där så kunde de ju inte annat göra än gå med bort till kyrkogården </s> <s> de fick ju också lov att småle för sig själva när de tänkte på a

You will now create a list of words from your string. You will consider that a space or a carriage return is an item separator

In [556]:
# Write your code here
words = re.findall(r"[<(\/)?s>\p{L}]+", corpus)

The five last lines of the corpus should like this:

In [557]:
print(words[-101:])
print(len(words))

['<s>', 'hon', 'hade', 'fått', 'större', 'kärlek', 'av', 'sina', 'föräldrar', 'än', 'någon', 'annan', 'han', 'visste', 'och', 'sådan', 'kärlek', 'måste', 'vändas', 'i', 'välsignelse', '</s>', '<s>', 'då', 'prästen', 'sade', 'detta', 'kom', 'alla', 'människor', 'att', 'se', 'bort', 'mot', 'klara', 'gulla', 'och', 'de', 'förundrade', 'sig', 'över', 'vad', 'de', 'såg', '</s>', '<s>', 'prästens', 'ord', 'tycktes', 'redan', 'ha', 'gått', 'i', 'uppfyllelse', '</s>', '<s>', 'där', 'stod', 'klara', 'fina', 'gulleborg', 'ifrån', 'skrolycka', 'hon', 'som', 'var', 'uppkallad', 'efter', 'själva', 'solen', 'vid', 'sina', 'föräldrars', 'grav', 'och', 'lyste', 'som', 'en', 'förklarad', '</s>', '<s>', 'hon', 'var', 'likaså', 'vacker', 'som', 'den', 'söndagen', 'då', 'hon', 'gick', 'till', 'kyrkan', 'i', 'den', 'röda', 'klänningen', 'om', 'inte', 'vackrare', '</s>']
1040958


### Counting unigrams and bigrams

Read and try programs to compute the frequency of unigrams and bigrams of the training set: [<a
            href="https://github.com/pnugues/ilppp/tree/master/programs/ch05/python">Program folder</a>].

#### Unigrams

In [558]:
def unigrams(words):
    frequency = {}
    for i in range(len(words)):
        if words[i] in frequency:
            frequency[words[i]] += 1
        else:
            frequency[words[i]] = 1
    return frequency

We compute the frequencies.

In [559]:
frequency = unigrams(words)
list(frequency.items())[:80]
print("sad",frequency["det"])

sad 21086


#### Bigrams

In [560]:
def bigrams(words):
    bigrams = []
    for i in range(len(words) - 1):
        bigrams.append((words[i], words[i + 1]))
    frequency_bigrams = {}
    for i in range(len(words) - 1):
        if bigrams[i] in frequency_bigrams:
            frequency_bigrams[bigrams[i]] += 1
        else:
            frequency_bigrams[bigrams[i]] = 1
    return frequency_bigrams

In [561]:
frequency_bigrams = bigrams(words)
list(frequency_bigrams.items())[:20]

[(('<s>', 'selma'), 8),
 (('selma', 'lagerlöf'), 11),
 (('lagerlöf', 'nils'), 1),
 (('nils', 'holgerssons'), 6),
 (('holgerssons', 'underbara'), 4),
 (('underbara', 'resa'), 4),
 (('resa', 'genom'), 6),
 (('genom', 'sverige'), 5),
 (('sverige', '</s>'), 17),
 (('</s>', '<s>'), 59046),
 (('<s>', 'första'), 11),
 (('första', 'bandet'), 1),
 (('bandet', 'bokutgåva'), 2),
 (('bokutgåva', 'albert'), 11),
 (('albert', 'bonniers'), 11),
 (('bonniers', 'förlag'), 11),
 (('förlag', 'stockholm'), 10),
 (('stockholm', '</s>'), 24),
 (('<s>', 'den'), 1375),
 (('den', 'kristliga'), 2)]

In the report, tell what is the possible number of bigrams and their real number? Explain why such a difference. What would be the possible number of 4-grams.

Propose a solution to cope with bigrams unseen in the corpus.

### Computing the likelihood of a sentence

#### Unigrams

Write a program to compute a sentence's probability using unigrams. You may find useful the dictionaries that we saw in the mutual information program: [<a href="https://github.com/pnugues/ilppp/tree/master/programs/ch05/python">Program folder</a>]. Your function will return the perplexity.

Your function should print and tabulate the results as in the examples below with the sentence _Det var en gång en katt som hette Nils_.

```
=====================================================
wi 	 C(wi) 	 #words 	 P(wi)
=====================================================
det 	 21108 	 1041631 	 0.0202643738521607
var 	 12090 	 1041631 	 0.01160679741674355
en 	 13514 	 1041631 	 0.01297388422579589
gång 	 1332 	 1041631 	 0.001278763784871994
en 	 13514 	 1041631 	 0.01297388422579589
katt 	 16 	 1041631 	 1.5360525944408337e-05
som 	 16288 	 1041631 	 0.015637015411407686
hette 	 97 	 1041631 	 9.312318853797554e-05
nils 	 87 	 1041631 	 8.352285982272032e-05
</s> 	 59047 	 1041631 	 0.056687060964967444
=====================================================
Prob. unigrams:	 5.361459667285409e-27
Geometric mean prob.: 0.0023600885848765307
Entropy rate:	 8.726943273141258
Perplexity:	 423.71290908655254
```

In [562]:
# Write your code
def unigram_lm(frequency, text):
    print("=====================================================")
    print("wi", "C(wi)", "#words", "P(wi)", sep = "\t")
    prob_uni = 1
    l = sum(frequency.values())
    for w in text:
        if w in frequency:
            c = frequency[w]
            prob = c/l
            prob_uni *= prob
            print(w, c, l, prob, sep = "\t")
    len_words = len(text)
    geo_mean = prob_uni**(1/len_words)
    entropy = -1/(len_words)*math.log2(prob_uni)
    perplexity = 1/geo_mean
    print("=====================================================")
    print("Prob. unigrams: ", prob_uni)
    print("Geometric mean prob.: ", str(geo_mean))
    print("Entropy rate: " + str(entropy))
    print("Perplexity: " + str(perplexity))

    return perplexity


In [563]:
sentence = 'det var en gång en katt som hette nils </s>'
sent_words = sentence.split()
sent_words

['det', 'var', 'en', 'gång', 'en', 'katt', 'som', 'hette', 'nils', '</s>']

In [564]:
perplexity_unigrams = unigram_lm(frequency, sent_words)

wi	C(wi)	#words	P(wi)
det	21086	1040958	0.020256340793768816
var	12074	1040958	0.011598930984727529
en	13481	1040958	0.012950570532144429
gång	1332	1040958	0.0012795905310300703
en	13481	1040958	0.012950570532144429
katt	16	1040958	1.5370456829190033e-05
som	16288	1040958	0.015647125052115456
hette	97	1040958	9.318339452696458e-05
nils	87	1040958	8.357685900872082e-05
</s>	59047	1040958	0.056723710274574
Prob. unigrams:  5.3572055679837745e-27
Geometric mean prob.:  0.0023599012545878913
Entropy rate: 8.72705779053971
Perplexity: 423.7465436555436


In [565]:
perplexity_unigrams = int(perplexity_unigrams)
perplexity_unigrams

423

#### Bigrams

Write a program to compute the sentence probability using bigrams. Your function will tabulate and print the results as below. It will return the perplexity.

```
=====================================================
wi 	 wi+1 	 Ci,i+1 	 C(i) 	 P(wi+1|wi)
=====================================================
<s>	 det 	 5672 	 59047 	 0.09605907158704083
det 	 var 	 3839 	 21108 	 0.1818741709304529
var 	 en 	 712 	 12090 	 0.058891645988420185
en 	 gång 	 706 	 13514 	 0.052242119283705785
gång 	 en 	 20 	 1332 	 0.015015015015015015
en 	 katt 	 6 	 13514 	 0.0004439840165754033
katt 	 som 	 2 	 16 	 0.125
som 	 hette 	 45 	 16288 	 0.002762770137524558
hette 	 nils 	 0 	 97 	 0.0 	 *backoff: 	 8.352285982272032e-05
nils 	 </s> 	 2 	 87 	 0.022988505747126436
=====================================================
Prob. bigrams:	 2.376007803503683e-19
Geometric mean prob.: 0.013727289294133601
Entropy rate:	 6.186809422848149
Perplexity:	 72.84759420254609
```

In [566]:
# Write your code
def bigram_lm(frequency, frequency_bigrams, text):
    print("=====================================================")
    print("wi", "wi+1", "Ci,i+1","C(i)", "P(wi+1|wi)", sep = "\t")
    prob_uni = 1
    for i in range(len(text) - 1):
        wi = text[i]
        wiplus = text[i+1]

        if (wi, wiplus) in frequency_bigrams:
            ciplus = frequency_bigrams[(wi,wiplus)]
        else: ciplus = 0

        ci = frequency[wi]

        if ciplus != 0:
            prob = ciplus/ci
        else: 
            prob = frequency[wiplus] / sum(frequency.values())
        
        prob_uni *= prob
        print(wi, wiplus, ciplus, ci, prob, sep="\t")

    len_words = len(text) - 1
    geo_mean = prob_uni**(1/len_words)
    entropy = (-1/(len_words))*math.log2(prob_uni)
    perplexity = 2**entropy
    
    print("=====================================================")
    print("Prob. unigrams: ", prob_uni)
    print("Geometric mean prob.: ", str(geo_mean))
    print("Entropy rate: " + str(entropy))
    print("Perplexity: " + str(perplexity))

    return perplexity

In [567]:
sentence2 = '<s> det var en gång en katt som hette nils </s>'
sent_words2 = sentence2.split()
sent_words2
perplexity_bigrams = bigram_lm(frequency, frequency_bigrams, sent_words2)

wi	wi+1	Ci,i+1	C(i)	P(wi+1|wi)
<s>	det	5672	59047	0.09605907158704083
det	var	3839	21086	0.1820639286730532
var	en	712	12074	0.058969686930594664
en	gång	706	13481	0.0523700022253542
gång	en	20	1332	0.015015015015015015
en	katt	6	13481	0.0004450708404421037
katt	som	2	16	0.125
som	hette	45	16288	0.002762770137524558
hette	nils	0	97	8.357685900872082e-05
nils	</s>	2	87	0.022988505747126436
Prob. unigrams:  2.3948602542817367e-19
Geometric mean prob.:  0.013738142525943445
Entropy rate: 6.1856692329085
Perplexity: 72.79004407703411


In [568]:
perplexity_bigrams = int(perplexity_bigrams)
perplexity_bigrams

72

In addition to this sentence, _Det var en gång en katt som hette Nils_, write three other sentences that will form your test set and run your programs on them. You will insert them in your report.

### Online prediction of words

You will now carry out an online prediction of words. You will consider two cases:
1. Prediction of the current word a user is typing;
2. Prediction of the next word.

Ideally, you would write a loop that reads the words and apply the models while typing. As the Jupyter labs are not designed for interactive input and output, we will simplify the experimental settings with constant strings at a given time of the input.  

We will assume the user is typing the phrase: _Det var en gång_. 

#### Trigrams

To have a more accurate prediction, you will use a trigram counting function. Program this function following the model of your bigram counting function.

In [569]:
# Write your code
# Det var en gång [(Det, var, en), (var, en, gång)]

def trigrams(words):
    trigrams = []
    for i in range(len(words) - 2):
        trigrams.append((words[i], words[i + 1], words[i+2]))
    frequency_trigrams = {}
    for i in range(len(words) - 2):
        if trigrams[i] in frequency_trigrams:
            frequency_trigrams[trigrams[i]] += 1
        else:
            frequency_trigrams[trigrams[i]] = 1
    return frequency_trigrams


In [570]:
frequency_trigrams = trigrams(words)
frequency_trigrams[('det', 'var', 'en')]

330

#### Prediction

The user starts typing _Det var en gång_. After the 2nd character, your program tries to help the user with suggested words.

In [571]:
starting_text = 'De'.lower()
starting_text

'de'

Write a program to rank the five first candidates at this point. Assign these predictions in a list that you will call `current_word_predictions_1`. Note that you are starting a sentence and you can then use the bigram frequencies. Write a sorting key that will enable you have a deterministic ranking of the words or bigrams with identical frequencies: When two words have the same frequency, you will sort them by alphabetic order. You can do this with a tuple.

In [572]:
cand_nbr = 5 

In [573]:
# Write your code here
list_cand = []
for (w1,w2) in frequency_bigrams:
    if(w1 == "<s>" and w2.startswith(starting_text)):
        list_cand.append(w2)

list_cand.sort(key = lambda x: frequency_bigrams[("<s>", x)], reverse=True)

current_word_predictions_1 = list_cand[:cand_nbr]

In [574]:
current_word_predictions_1

['det', 'de', 'den', 'detta', 'denna']

Let us now suppose that the user has typed: _Det var en_. After detecting a space, your program starts predicting a next possible word.

In [575]:
current_text = "Det var en ".lower()
current_text

'det var en '

Tokenize this text and return a list of tokens. Call it `tokens`.

In [576]:
# Write your code here
tokens = tokenize(current_text)

In [577]:
tokens

['det', 'var', 'en']

Write a program to propose the five next possible words ranked by frequency using a trigram model. Assign these predictions to a variable that you will call `next_word_predictions`. Write a sorting key that will enable you have a deterministic ranking of the words or bigrams with identical frequencies: When two words have the same frequency, you will sort them by alphabetic order. You can do this with a tuple.

In [578]:
# Write your code here
list_cand = []
for (w1,w2,w3) in frequency_trigrams:
    if w1 == tokens[1] and w2 == tokens[2]:
        list_cand.append(w3)
list_cand.sort()
list_cand.sort(key=lambda x: frequency_trigrams[(tokens[1],tokens[2],x)], reverse = True)
next_word_predictions = list_cand[:cand_nbr]

In [246]:
next_word_predictions

['stor', 'liten', 'gammal', 'god', 'sådan']

Finally, let us suppose that the user has typed _Det var en g_, rank the five possible candidates. Assign these predictions in a list that you will call `current_word_predictions_2`

In [579]:
current_text = "Det var en g".lower()

In [580]:
# Write your code here
list_cand = []
for (w1,w2,w3) in frequency_trigrams:
    if w1 == tokens[1] and w2 == tokens[2] and w3.startswith('g'):
        list_cand.append(w3)
list_cand.sort()
list_cand.sort(key=lambda x: (frequency_trigrams[(tokens[1],tokens[2],x)]), reverse = True)
current_word_predictions_2 = list_cand[:cand_nbr]


In [249]:
current_word_predictions_2

['gammal', 'god', 'gång', 'ganska', 'glädje']

## Checked answers

The automatic marking system will check these answers: `(perplexity_unigrams, perplexity_bigrams, current_word_predictions_1, next_word_predictions, current_word_predictions_2)`

In [581]:
(perplexity_unigrams, perplexity_bigrams, current_word_predictions_1, next_word_predictions, current_word_predictions_2)

(423,
 72,
 ['det', 'de', 'den', 'detta', 'denna'],
 ['stor', 'liten', 'gammal', 'god', 'sådan'],
 ['gammal', 'god', 'gång', 'ganska', 'glädje'])

## Submission

When you have written all the code and run all the cells, fill in your ID and as well as the name of the notebook.

In [53]:
STIL_ID = ["student1", "student2"] # Write your stil ids as a list
CURRENT_NOTEBOOK_PATH = os.path.join(os.getcwd(), 
                                     "2-language_models_solution.ipynb") # Write the name of your notebook

The submission code will send your answer. It consists of the perplexities and predictions.

In [54]:
import json
ANSWER = json.dumps({'perplexity_unigrams': perplexity_unigrams,
                     'perplexity_bigrams': perplexity_bigrams,
                     'current_word_predictions_1': current_word_predictions_1,
                     'next_word_predictions': next_word_predictions,
                     'current_word_predictions_2': current_word_predictions_2})
ANSWER

'{"perplexity_unigrams": 423, "perplexity_bigrams": 72, "current_word_predictions_1": ["det", "de", "den", "detta", "denna"], "next_word_predictions": ["stor", "liten", "gammal", "god", "s\\u00e5dan"], "current_word_predictions_2": ["gammal", "god", "g\\u00e5ng", "ganska", "gl\\u00e4dje"]}'

The checking script will accept perplexities with a margin, like for instance:
```
(421, 72, ['det', 'de', 'den', 'detta', 'denna'], ['stor', 'liten', 'gammal', 'god', 'sådan'], ['gammal', 'god', 'gång', 'ganska', 'glädje'])
```

Now the moment of truth:
1. Save your notebook and
2. Run the cells below

In [55]:
SUBMISSION_NOTEBOOK_PATH = CURRENT_NOTEBOOK_PATH + ".submission.bz2"

In [56]:
ASSIGNMENT = 2
API_KEY = "f581ba347babfea0b8f2c74a3a6776a7"

# Copy and compress current notebook
with bz2.open(SUBMISSION_NOTEBOOK_PATH, mode="wb") as fout:
    with open(CURRENT_NOTEBOOK_PATH, "rb") as fin:
        fout.write(fin.read())

In [57]:
res = requests.post("https://vilde.cs.lth.se/edan20checker/submit", 
                    files={"notebook_file": open(SUBMISSION_NOTEBOOK_PATH, "rb")}, 
                    data={
                        "stil_id": STIL_ID,
                        "assignment": ASSIGNMENT,
                        "answer": ANSWER,
                        "api_key": API_KEY,
                    },
                   verify=True)

# from IPython.display import display, JSON
res.json()

{'msg': None,
 'status': 'correct',
 'signature': 'ef5370e5c56f10168ae79f3f8b2e09b2435b9c9b70c41e6b4313157b7dbff43d227c359e6774c5c9163347f3e1a6a0a1cb6301e80a1f7b2dfc647c15577c1909',
 'submission_id': '08986bf2-c8d4-40cd-84c8-9e5f84594908'}

## Turning in your assignment

Now your are done with the program. To complete this assignment, you will:
1. Write a short individual report on your program. You will include the regular expression you use to segment the text and the unigram and bigram tables for three sentences and _Det var en gång en katt som hette Nils_.
2. Execute the Jupyter notebook by Peter Norvig here: <a href="http://nbviewer.jupyter.org/url/norvig.com/ipython/How%20to%20Do%20Things%20with%20Words.ipynb">https://nbviewer.jupyter.org/url/norvig.com/ipython/How to Do Things with Words.ipynb</a>. Just run all the cells and be sure that you understand the code. You will find the data here: <a href="http://norvig.com/ngrams/">http://norvig.com/ngrams/</a>.
3. In your report, after the description of your program, you will describe one experiment with Norvig's notebook and a long string of words your will create yourself or copy from a text you like. You will remove all the punctuation and white spaces from this string. You will set this string in lowercase letters. You will just add a cell at the end of Sect. 7 in Norvig's notebook, where you will use your string and run the notebook cell with the <tt>segment()</tt> and <tt>segment2()</tt> functions. You will comment the segmentation results you obtain with the unigram and bigram models.

Submit your report as well as your notebook (for archiving purposes) to Canvas: https://canvas.education.lu.se/. To write your report, you can either
1. Write directly your text in Canvas, or
2. Use Latex and Overleaf (www.overleaf.com). This will probably help you structure your text. You will then upload a PDF file in Canvas.

The submission deadline is September 23, 2022. You will have only three submission attempts. The deadline for the second and third ones are one week after you are noticed of your result.